## Usando RNNs para classificar sentimento em dados do IMDB
Neste exercício, treinaremos uma RNN "vanilla" para prever o sentimento em avaliações do IMDB. Nossos dados consistem em 25000 sequências de treinamento e 25000 sequências de teste. O resultado é binário (positivo/negativo) e ambos os resultados são igualmente representados tanto no conjunto de treinamento quanto no conjunto de teste.

O Keras fornece uma interface conveniente para carregar os dados e codificar imediatamente as palavras em inteiros (com base nas palavras mais comuns). Isso nos poupará muito do trabalho pesado que geralmente está envolvido ao trabalhar com texto bruto.

Vamos percorrer a preparação dos dados e a construção de um modelo RNN. Então será sua vez de construir seus próprios modelos (e preparar os dados como achar melhor).

In [ ]:
from __future__ import print_function
import tensorflow.keras as keras
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding
from tensorflow.keras.layers import SimpleRNN
from tensorflow.keras.datasets import imdb
from tensorflow.keras import initializers

In [ ]:
max_features = 20000  # Usado no carregamento dos dados, seleciona as palavras mais comuns (max_features)
maxlen = 30  # comprimento máximo de uma sequência - truncar após isso
batch_size = 32

In [ ]:
## Carregar os dados. A função tokeniza automaticamente o texto em inteiros distintos
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'sequências de treino')
print(len(x_test), 'sequências de teste')

In [ ]:
# Isso preenche (ou trunca) as sequências para que tenham o comprimento máximo
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('forma de x_train:', x_train.shape)
print('forma de x_test:', x_test.shape)

In [ ]:
x_train[123,:]  # Aqui está como uma sequência de exemplo se parece

## Camadas Keras para RNNs (Vanilla)

Neste exercício, não usaremos vetores de palavras pré-treinados. Em vez disso, aprenderemos um embedding como parte da Rede Neural. Isso é representado pela Camada de Embedding abaixo.

### Camada de Embedding
`keras.layers.embeddings.Embedding(input_dim, output_dim, embeddings_initializer='uniform', embeddings_regularizer=None, activity_regularizer=None, embeddings_constraint=None, mask_zero=False, input_length=None)`

- Esta camada mapeia cada inteiro em um vetor de palavra (denso) distinto de comprimento `output_dim`.
- Pode-se pensar nisso como aprender um embedding de vetor de palavra "em tempo real" em vez de usar um mapeamento existente (como GloVe)
- O `input_dim` deve ser o tamanho do vocabulário.
- O `input_length` especifica o comprimento das sequências que a rede espera.

### Camada SimpleRNN
`keras.layers.recurrent.SimpleRNN(units, activation='tanh', use_bias=True, kernel_initializer='glorot_uniform', recurrent_initializer='orthogonal', bias_initializer='zeros', kernel_regularizer=None, recurrent_regularizer=None, bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, recurrent_constraint=None, bias_constraint=None, dropout=0.0, recurrent_dropout=0.0)`

- Esta é a RNN básica, onde a saída também é alimentada de volta como o "estado oculto" para a próxima iteração.
- O parâmetro `units` fornece a dimensionalidade da saída (e, portanto, do estado oculto). Note que tipicamente haverá outra camada após a RNN mapeando a saída (RNN) para a saída da rede. Então devemos pensar neste valor como a dimensionalidade desejada do estado oculto e não necessariamente a saída desejada da rede.
- Lembre-se de que existem dois conjuntos de pesos, um para a fase "recorrente" e outro para a fase "kernel". Eles podem ser configurados separadamente em termos de sua inicialização, regularização, etc.






In [ ]:
## Vamos construir uma RNN

rnn_hidden_dim = 5
word_embedding_dim = 50
model_rnn = Sequential()
model_rnn.add(Embedding(max_features, word_embedding_dim))  # Esta camada pega cada inteiro na sequência e o incorpora em um vetor de 50 dimensões
model_rnn.add(SimpleRNN(rnn_hidden_dim,
                    kernel_initializer=initializers.RandomNormal(stddev=0.001),
                    recurrent_initializer=initializers.Identity(gain=1.0),
                    activation='relu',
                    input_shape=x_train.shape[1:]))

model_rnn.add(Dense(1, activation='sigmoid'))

In [ ]:
## Note que a maioria dos parâmetros vem da camada de embedding
model_rnn.summary()

In [ ]:
rmsprop = keras.optimizers.RMSprop(learning_rate=.0001)

model_rnn.compile(loss='binary_crossentropy',
              optimizer=rmsprop,
              metrics=['accuracy'])

In [ ]:
model_rnn.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=10,
          validation_data=(x_test, y_test))

In [ ]:
score, acc = model_rnn.evaluate(x_test, y_test,
                            batch_size=batch_size)
print('Pontuação no teste:', score)
print('Precisão no teste:', acc)

## Exercício
### Sua Vez

Agora faça você mesmo:
- Prepare os dados para usar sequências de comprimento 80 em vez de comprimento 30. Isso melhorou o desempenho?
- Tente diferentes valores de "max_features". Você consegue melhorar o desempenho?
- Tente tamanhos menores e maiores da dimensão oculta da RNN. Como isso afeta o desempenho do modelo? Como afeta o tempo de execução?

In [ ]:
# Escreva seu código aqui